 - Same as base replication, but use ordinal regression.
 - https://pythonhosted.org/mord/
 - Regression technique for predicting ranking
 - Since we are creating a portfolio using top quintile - bottom quintile, we are more interested in the correct rank than the actual value, so thought, might as well give it a shot.
 - A multiclass classification using softmax as loss function to maximize accuracy, just goes by true/false classification but one might use a loss function that understands how the classes relate, and penalizes an off-by-1 quintile error less than off-by-2 etc.
 - Doesn't seem to outperform the base replication

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import time 
import copy
import random
from itertools import product

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy numpy warnings

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.linear_model import LinearRegression, Lasso, lasso_path, lars_path, LassoLarsIC
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import mord

import ffn
%matplotlib inline

import plotly as py
# print (py.__version__) # requires version >= 1.9.0
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

random.seed(1764)
np.random.seed(1764)


In [2]:
print("Loading data...")
data = pd.read_csv("30_Industry_Portfolios.csv")
data = data.set_index('yyyymm')
industries = list(data.columns)
# map industry names to col nums
ind_reverse_dict = dict([(industries[i], i) for i in range(len(industries))])

rfdata = pd.read_csv("F-F_Research_Data_Factors.csv")
rfdata = rfdata.set_index('yyyymm')
data['rf'] = rfdata['RF']

# subtract risk-free rate
# create a response variable led by 1 period to predict
for ind in industries:
    data[ind] = data[ind] - data['rf']

#for ind in industries:
#    data[ind+".3m"] = pd.rolling_mean(data[ind],3)
    
#for ind in industries:
#    data[ind+".6m"] = pd.rolling_mean(data[ind],6)

#for ind in industries:
#    data[ind+".12m"] = pd.rolling_mean(data[ind],12)
    
for ind in industries:
    data[ind+".lead"] = data[ind].shift(-1)

data = data.loc[data.index[data.index > 195911]]
data = data.drop(columns=['rf'])    
data = data.dropna(axis=0, how='any')

nresponses = len(industries)
npredictors = data.shape[1]-nresponses

predictors = list(data.columns[:npredictors])
predictor_reverse_dict = dict([(predictors[i], i) for i in range(len(predictors))])

responses = list(data.columns[-nresponses:])
response_reverse_dict = dict([(responses[i], i) for i in range(len(responses))])

print(data.shape)

data[['Food', 'Food.lead']]


Loading data...
(697, 60)


,Food,Food.lead
yyyymm,,
195912,2.01,-4.49
196001,-4.49,3.35
196002,3.35,-1.67
196003,-1.67,1.17
196004,1.17,8.20
196005,8.20,5.39
196006,5.39,-2.11
196007,-2.11,4.57
196008,4.57,-3.88


In [3]:
# exclude 2017 and later to tie to paper
data = data.loc[data.index[data.index < 201701]]
data = data.loc[data.index[data.index > 195911]]
data


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
195912,2.01,0.35,-3.02,1.64,7.29,0.67,1.87,-1.97,3.08,0.74,...,0.62,-6.18,-7.93,-9.41,-4.31,-5.33,-6.09,-10.08,-4.68,-3.98
196001,-4.49,-5.71,-2.05,1.21,-5.47,-7.84,-8.53,-6.68,-10.03,-4.77,...,8.07,9.13,5.09,3.00,-0.94,1.42,4.00,1.81,-0.98,6.32
196002,3.35,-2.14,2.27,4.23,2.39,9.31,1.44,-0.02,-0.74,0.32,...,-0.21,-0.31,3.34,-2.43,-4.99,-1.37,-0.13,-3.88,0.05,-2.43
196003,-1.67,-2.94,-0.18,-0.65,2.18,-0.56,-2.59,1.26,-2.75,-6.79,...,-1.24,7.14,1.77,0.41,-2.13,0.45,-0.53,8.86,-0.64,0.55
196004,1.17,-2.16,1.35,6.46,-1.17,-1.27,0.21,1.49,-5.53,-1.10,...,3.05,-1.75,11.90,2.85,0.90,1.65,3.11,0.80,-0.45,1.02
196005,8.20,-0.52,2.44,7.28,11.67,7.74,1.74,13.50,3.40,2.10,...,-0.58,-8.07,2.39,3.50,2.17,5.96,3.41,1.03,3.72,6.41
196006,5.39,0.47,4.73,2.24,0.02,6.38,-1.59,-0.40,0.45,4.04,...,-0.03,2.84,-2.02,-4.10,-3.11,-6.16,-2.99,-1.25,0.09,-5.95
196007,-2.11,-0.79,4.60,-4.72,0.23,-0.60,-1.10,-3.99,-6.80,-3.14,...,6.94,5.69,2.71,1.18,1.98,4.51,2.85,2.05,3.47,3.48
196008,4.57,3.24,5.20,7.16,3.63,5.09,3.34,2.29,1.17,-0.84,...,-6.07,-3.53,-7.61,-7.37,-7.07,-8.44,-8.57,-1.90,-5.78,-4.21


In [4]:
desc = data.describe()
desc
# min, max line up with Table 1

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
count,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,...,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000,685.000000
mean,0.690715,0.710613,0.982321,0.701708,0.528277,0.554190,0.669460,0.650905,0.519781,0.667416,...,0.520847,0.694234,0.584175,0.511241,0.582088,0.625562,0.662219,0.702730,0.609810,0.385620
std,4.339811,5.090215,6.061582,7.180918,5.809314,4.759874,6.386027,4.928072,5.518477,7.022552,...,4.628520,6.527984,6.738979,5.055314,5.739306,5.605317,5.349341,6.104515,5.411766,5.815446
min,-18.150000,-20.190000,-25.320000,-33.400000,-26.560000,-22.240000,-31.500000,-21.060000,-28.600000,-33.110000,...,-16.440000,-28.670000,-32.070000,-27.740000,-28.500000,-29.250000,-29.740000,-31.890000,-22.530000,-28.090000
25%,-1.640000,-2.100000,-2.780000,-3.490000,-2.690000,-2.110000,-2.810000,-2.240000,-2.800000,-3.200000,...,-2.110000,-3.090000,-3.290000,-2.430000,-2.780000,-2.570000,-2.430000,-2.940000,-2.420000,-2.990000
50%,0.740000,0.710000,1.280000,0.890000,0.510000,0.750000,0.690000,0.750000,0.670000,0.630000,...,0.610000,0.970000,0.560000,0.690000,0.860000,0.940000,0.470000,1.030000,0.820000,0.470000
75%,3.120000,3.660000,4.640000,5.310000,3.720000,3.550000,4.310000,3.560000,3.760000,4.490000,...,3.360000,4.290000,4.590000,3.460000,4.060000,3.880000,4.000000,4.330000,4.000000,4.200000
max,19.890000,25.510000,32.380000,34.520000,33.130000,18.220000,31.790000,29.010000,21.680000,59.030000,...,21.220000,23.380000,24.660000,21.000000,18.500000,17.530000,26.490000,27.380000,20.590000,19.960000


In [5]:
# annualized returns don't match Table 1, oddly
# geometric mean, annualized
pd.DataFrame((np.prod(data/100 + 1)**(12.0/len(data))-1)[:30], columns=['Mean Ann. Return'])

,Mean Ann. Return
Food,0.074020
Beer,0.072005
Smoke,0.100147
Games,0.054031
Books,0.043953
Hshld,0.054098
Clths,0.057170
Hlth,0.065463
Chems,0.044917
Txtls,0.051888


In [6]:
# try this way, arithmetic mean then annualize (not very correct)
#print(pd.DataFrame(((desc.loc['mean']/100+1)**12-1)[:30]))
#nope

# same
pd.DataFrame(((1 + np.mean(data, axis=0)/100)**12 -1)[:30], columns=['Mean Ann. Return'])

,Mean Ann. Return
Food,0.086108
Beer,0.088687
Smoke,0.124460
Games,0.087532
Books,0.065268
Hshld,0.068568
Clths,0.083360
Hlth,0.080966
Chems,0.064188
Txtls,0.083096


In [7]:
#annualized volatility 
pd.DataFrame((desc.loc['std']*np.sqrt(12))[:30].round(2))
# lines up with table 1

,std
Food,15.03
Beer,17.63
Smoke,21.00
Games,24.88
Books,20.12
Hshld,16.49
Clths,22.12
Hlth,17.07
Chems,19.12
Txtls,24.33


In [8]:
# Run LASSO, then OLS on selected variables

# skip last row to better match published r-squared
# looks like they forecast actuals 1960-2016 using 1959m12 to 2016m11
# not exact matches to Table 2 R-squared but almost within rounding error 
X = data.values[:-1,:npredictors]
Y = data.values[:-1,-nresponses:]
nrows = X.shape[0]
X.shape

(684, 30)

In [9]:
def subset_selection(X, Y, model_aic, verbose=False):
    
    global responses
    global response_reverse_dict
    global predictors
    global predictor_reverse_dict
    
    coef_dict = {}
    for response_index, response in enumerate(responses):
        y = Y[:,response_reverse_dict[response]]
        
        model_aic.fit(X, y)

        coef_dict[response] = [predstr for i, predstr in enumerate(predictors) if model_aic.coef_[i] !=0]
        #y_response = model_aic.responseict(X)
        # print ("In-sample LASSO R-squared: %.6f" % r2_score(y, y_response))
        if verbose:
            print("LASSO variables selected for %s: " % response)
            print(coef_dict[response])
        
        if not coef_dict[response]:
            if verbose:
                print("No coefs selected for " + response + ", using all")
                print("---")
            coef_dict[response] = predictors            
        # fit OLS vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        if verbose:
            print("Running OLS for " + response + " against " + str(coef_dict[response]))
            # col nums of selected responses
            predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
            model_ols = LinearRegression()
            model_ols.fit(X[:, predcols], y)
            y_pred = model_ols.predict(X[:, predcols])
            print ("In-sample OLS R-squared: %.2f%%" % (100 * r2_score(y, y_pred)))
            print("---")
            
    return coef_dict

coef_dict = subset_selection(X, Y, LassoLarsIC(criterion='aic'), verbose=True)

# These subsets line up closely with Table 2
# except Clths, Whlsl, we get different responses

LASSO variables selected for Food.lead: 
['Clths', 'Coal', 'Util', 'Rtail']
Running OLS for Food.lead against ['Clths', 'Coal', 'Util', 'Rtail']
In-sample OLS R-squared: 2.24%
---
LASSO variables selected for Beer.lead: 
['Food', 'Clths', 'Coal']
Running OLS for Beer.lead against ['Food', 'Clths', 'Coal']
In-sample OLS R-squared: 2.52%
---
LASSO variables selected for Smoke.lead: 
['Txtls', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
Running OLS for Smoke.lead against ['Txtls', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
In-sample OLS R-squared: 6.55%
---
LASSO variables selected for Games.lead: 
['Books', 'Clths', 'Coal', 'Fin']
Running OLS for Games.lead against ['Books', 'Clths', 'Coal', 'Fin']
In-sample OLS R-squared: 5.05%
---
LASSO variables selected for Books.lead: 
['Games', 'Books', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Fin']
Running OLS for Books.lead against ['Games', 'Books', 'Coal'

In [10]:
# same predictors selected for all but 2 response vars
# use predictors from paper to match results
if True: # turn off/on
    coef_dict = {}
    coef_dict['Food.lead'] = ['Clths', 'Coal', 'Util', 'Rtail']
    coef_dict['Beer.lead'] = ['Food', 'Clths', 'Coal']
    coef_dict['Smoke.lead'] = ['Txtls', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
    coef_dict['Games.lead'] = ['Books', 'Clths', 'Coal', 'Fin']
    coef_dict['Books.lead'] = ['Games', 'Books', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Fin']
    coef_dict['Hshld.lead'] = ['Clths', 'Coal', 'Rtail']
    coef_dict['Clths.lead'] = ['Books', 'Clths', 'Chems', 'Steel', 'ElcEq', 'Carry',  'Coal', 'Oil', 'Util','Telcm', 'Servs', 'BusEq', 'Rtail']
    # Running OLS for Clths against ['Clths', 'Coal', 'Oil', 'Servs', 'Rtail']
    coef_dict['Hlth.lead'] = ['Books', 'Mines', 'Coal', 'Util']
    coef_dict['Chems.lead'] = ['Clths']
    coef_dict['Txtls.lead'] = ['Clths', 'Autos', 'Coal', 'Oil', 'Rtail', 'Fin']
    coef_dict['Cnstr.lead'] = ['Clths', 'Coal', 'Oil', 'Util', 'Trans', 'Rtail', 'Fin']
    coef_dict['Steel.lead'] = ['Fin']
    coef_dict['FabPr.lead'] = ['Trans', 'Fin']
    coef_dict['ElcEq.lead'] = ['Fin']
    coef_dict['Autos.lead'] = ['Hshld', 'Clths', 'Coal', 'Oil', 'Util', 'BusEq', 'Rtail', 'Fin']
    coef_dict['Carry.lead'] = ['Trans']
    coef_dict['Mines.lead'] = []
    coef_dict['Coal.lead'] = ['Beer', 'Smoke', 'Books', 'Autos', 'Coal', 'Oil', 'Paper', 'Rtail']
    coef_dict['Oil.lead'] = ['Beer', 'Hlth', 'Carry']
    coef_dict['Util.lead'] = ['Food', 'Beer', 'Smoke', 'Hshld', 'Hlth', 'Cnstr', 'FabPr', 'Carry', 'Mines', 'Oil', 'Util', 'Telcm', 'BusEq', 'Whlsl', 'Fin', 'Other']
    coef_dict['Telcm.lead'] = ['Beer', 'Smoke', 'Books', 'Hshld', 'Cnstr', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Meals', 'Fin']
    coef_dict['Servs.lead'] = ['Smoke', 'Books', 'Steel', 'Oil', 'Util', 'Fin']
    coef_dict['BusEq.lead'] = ['Smoke', 'Books', 'Util']
    coef_dict['Paper.lead'] = ['Clths', 'Coal', 'Oil', 'Rtail', 'Fin']
    coef_dict['Trans.lead'] = ['Fin']
    coef_dict['Whlsl.lead'] = ['Food', 'Beer', 'Smoke', 'Books', 'Hlth', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Fin', 'Other']
    # Running OLS for Whlsl against ['Food', 'Smoke', 'Books', 'Carry', 'Coal', 'Oil', 'Util', 'Servs', 'Fin', 'Other']
    coef_dict['Rtail.lead'] = ['Rtail']
    coef_dict['Meals.lead'] = ['Smoke', 'Books', 'Clths', 'Steel', 'Carry', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Meals', 'Fin']
    coef_dict['Fin.lead'] = ['Fin']
    coef_dict['Other.lead'] = ['Clths', 'Fin']


In [11]:
def predict_with_subsets(X, Y, create_model, coef_dict, verbose=False):
    """similar output as subset_selection, but pass a model function and a dictionary, compute avg R-squared"""
    global responses
    global response_reverse_dict
    
    model = create_model()
    
    scores = []
    for response in responses:
        y = Y[:,response_reverse_dict[response]]

#        print("LASSO variables selected for %s: " % pred)
#        print(coef_dict[pred])
        
        if not coef_dict[response]:
            if verbose:
                print("No coefs selected for " + response)
 #           print("---")
            continue
        # fit model vs. selected vars, better fit w/o LASSO penalties
        # in-sample R-squared using LASSO coeffs
        #print("Running model for " + pred + " against " + str(coef_dict[pred]))
        # col nums of selected predictors
        predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
        model.fit(X[:, predcols], y)
        y_pred = model.predict(X[:, predcols])
        score = r2_score(y, y_pred)
        scores.append(score)
        if verbose:
            print ("In-sample R-squared: %.4f%% for %s against %s" % (score*100, response, str(coef_dict[response])))
#        print("---")
    
    if verbose:
        print("Mean R-squared: %.4f%%" % (100 * np.mean(np.array(scores))))
    return np.mean(np.array(scores))
    

predict_with_subsets(X, Y, LinearRegression, coef_dict, verbose=True)


In-sample R-squared: 2.2426% for Food.lead against ['Clths', 'Coal', 'Util', 'Rtail']
In-sample R-squared: 2.5175% for Beer.lead against ['Food', 'Clths', 'Coal']
In-sample R-squared: 6.5482% for Smoke.lead against ['Txtls', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'Paper', 'Trans', 'Fin']
In-sample R-squared: 5.0487% for Games.lead against ['Books', 'Clths', 'Coal', 'Fin']
In-sample R-squared: 6.3011% for Books.lead against ['Games', 'Books', 'Coal', 'Oil', 'Util', 'Servs', 'BusEq', 'Rtail', 'Fin']
In-sample R-squared: 2.9707% for Hshld.lead against ['Clths', 'Coal', 'Rtail']
In-sample R-squared: 7.8178% for Clths.lead against ['Books', 'Clths', 'Chems', 'Steel', 'ElcEq', 'Carry', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Rtail']
In-sample R-squared: 2.6789% for Hlth.lead against ['Books', 'Mines', 'Coal', 'Util']
In-sample R-squared: 0.7764% for Chems.lead against ['Clths']
In-sample R-squared: 7.9132% for Txtls.lead against ['Clths', 'Autos', 'Coal', 'Oil'

0.03862278631691256

In [12]:
coef_dict_all = {}
for response in responses:
    coef_dict_all[response] = predictors
predict_with_subsets(X, Y, LinearRegression, coef_dict_all, verbose=True)


In-sample R-squared: 5.1275% for Food.lead against ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
In-sample R-squared: 5.0555% for Beer.lead against ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
In-sample R-squared: 7.4780% for Smoke.lead against ['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other']
In-sample R-squared: 7.5633% for Games.lead against ['Food',

0.0663748688823735

In [13]:
def fit_predict(X, Y, create_model, coef_dict=None, npredict=1):
    """for backtest, train model using Y_list v. X using n-npredict rows
    generate npredict prediction Y_list using last npredict rows of X
    if npredict=1, fit using n-1 rows, return prediction using X for final row
    if npredict=12, fit using n-12 rows, return prediction using X for final 12 rows
    coef_dict = None -> perform subset selection at each timestep using subset_selection()
    coef_dict = 'all' -> use all predictors"""
    
    global responses
    global response_reverse_dict
        
    # keep last row to predict against
    X_predict = X[-npredict:]
    nrows, ncols = X.shape
    X_predict = X_predict.reshape(npredict,ncols)
    # fit on remaining rows
    X_fit = X[:-npredict]
    Y_fit = Y[:-npredict]

    # if no coef_dict select predictors into coef_dict
    if coef_dict is None:
        coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'))
    # if coef_dict == "all" use all predictors for each response        
    elif coef_dict == 'all':
        coef_dict = {}
        for response in responses:
            coef_dict[response]=predictors

    predictions = []
    for response in responses:
        if not coef_dict[response]:
            predictions.append([np.nan]*npredict)
            continue
        # column indexes to fit against each other
        predcols = [predictor_reverse_dict[predstr] for predstr in coef_dict[response]]
        responsecol = response_reverse_dict[response]
        X_temp = X_fit[:, predcols]
        y_temp = Y_fit[:,responsecol]
        print("fitting on " + str(X_temp.shape))
        print(" vs " + str(y_temp.shape))
        model = create_model()
        model.fit(X_temp, y_temp)
        print("predicting on" + str(X_predict[:, predcols].shape))
        y_pred = model.predict(X_predict[:,predcols])        
        predictions.append(y_pred)
        
    return np.array(predictions).transpose()

#    return np.argsort(predictions)

X = data.values[:,:npredictors]
Y = data.values[:, -nresponses:]
print(X[684])
print(Y[684])
predictions = fit_predict(X[:685], Y[:685], LinearRegression, coef_dict, 1)
print(predictions)

[ 4.43  3.    5.39 -3.36  1.98  1.43 -0.44  0.82  0.32 -1.27  0.16 -2.14
 -0.15 -0.4   4.16  1.4  -1.8  -9.68  2.17  3.61  4.65 -0.19  2.07  1.86
  0.84  2.34 -0.98  0.58  3.8   2.57]
[ 9.500e-01 -1.020e+00  5.610e+00  4.840e+00  1.600e+00  2.720e+00
 -1.000e-02  2.170e+00  3.790e+00  6.900e+00  3.850e+00  3.600e+00
  4.640e+00  4.350e+00  5.180e+00  1.950e+00  1.237e+01 -5.360e+00
 -4.370e+00  1.050e+00  3.360e+00  5.450e+00  3.200e+00  2.280e+00
  1.700e+00  1.690e+00  9.300e-01  7.100e-01  5.600e-01 -8.700e-01]
fitting on (684, 4)
 vs (684,)
predicting on(1, 4)
fitting on (684, 3)
 vs (684,)
predicting on(1, 3)
fitting on (684, 11)
 vs (684,)
predicting on(1, 11)
fitting on (684, 4)
 vs (684,)
predicting on(1, 4)
fitting on (684, 9)
 vs (684,)
predicting on(1, 9)
fitting on (684, 3)
 vs (684,)
predicting on(1, 3)
fitting on (684, 13)
 vs (684,)
predicting on(1, 13)
fitting on (684, 4)
 vs (684,)
predicting on(1, 4)
fitting on (684, 1)
 vs (684,)
predicting on(1, 1)
fitting on (684, 

In [14]:
# first iteration will train up to including 196911
# will use 196912 to predict 197001
# 1970101 will be first month of performance to use
# train on first 121 months up to 196912 (0:120), put first prediction in P[121] (122nd row)
# first month of performance will be 197002
FIRST_TRAIN_MONTHS = 121
FIRST_PREDICT_MONTH = FIRST_TRAIN_MONTHS # This is stupid but keeps my head straight

print(X[FIRST_TRAIN_MONTHS])
print(data.iloc[FIRST_TRAIN_MONTHS][:30])

[ -3.34  -1.95  -7.59  -7.76 -12.05  -7.5   -5.69  -7.71  -7.37  -5.26
  -9.84  -6.31  -7.15  -6.89  -9.35 -12.49  -2.34  -0.77 -12.16  -4.83
  -3.16 -11.17  -9.73  -8.89  -8.17  -8.28  -6.31 -13.12  -9.78  -6.2 ]
Food     -3.34
Beer     -1.95
Smoke    -7.59
Games    -7.76
Books   -12.05
Hshld    -7.50
Clths    -5.69
Hlth     -7.71
Chems    -7.37
Txtls    -5.26
Cnstr    -9.84
Steel    -6.31
FabPr    -7.15
ElcEq    -6.89
Autos    -9.35
Carry   -12.49
Mines    -2.34
Coal     -0.77
Oil     -12.16
Util     -4.83
Telcm    -3.16
Servs   -11.17
BusEq    -9.73
Paper    -8.89
Trans    -8.17
Whlsl    -8.28
Rtail    -6.31
Meals   -13.12
Fin      -9.78
Other    -6.20
Name: 197001, dtype: float64


In [15]:
# predict all months
# initial train_months = 120 -> train first model on 120 rows
# first prediction will be in P[120] (121st row)
# step = 6 -> predict following 6 rows, then step forward 6 months at a time
# initialize global predictions matrix P

def run_backtest(X, Y, create_model, coef_dict=None, initial_train_months=0, 
                 minmaxscale=False, standardscale=False, step=1, verbose=False):
    global P
    P = np.zeros_like(Y)

    count = 0
    nrows, ncols = Y.shape
    
    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = MinMaxScaler().fit_transform(Yscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = StandardScaler().fit_transform(Yscale.transpose()).transpose()
        print("using StandardScaler")
        
    # create list of steps
    month_indexes = list(range(initial_train_months+step, nrows, step))
    # force an iteration for last (nrows % step) rows
    if month_indexes[-1] < nrows: 
        month_indexes.append(nrows)
    if verbose:
        print("Initial train rows = %d" % initial_train_months)
        print ("Steps: " + str(month_indexes))

    for month_index in month_indexes:
        # if month_index =121 and step =1
        # train on 0:119 (120 rows)
        # fit on 120 (121st row)
        # if month_index = 1000 and step = 10
        # fit on 0:989, predict 990:999
        if verbose:
            print("Training on first %d rows (%d:%d), putting prediction in rows %s" % (month_index-step, 0, month_index-step-1, str(range(month_index-step,month_index))))
        predictions = fit_predict(Xscale[:month_index], 
                                  Yscale[:month_index], 
                                  create_model,
                                  coef_dict,
                                  step)
        #990
        first_pred_row = month_index-step
        for row_index in range(step):
            P[first_pred_row+row_index] = predictions[row_index]
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training %d of %d" % (time.strftime("%H:%M:%S"), count, (X.shape[0]-initial_train_months+1)))
            sys.stdout.flush()
    print("")

    msetemp = (P[initial_train_months:]-Yscale[initial_train_months:])**2
    #remove nans
    msetemp = msetemp[~np.isnan(msetemp)]
    mse = np.mean(msetemp)
    print("MSE across all predictions: %.4f" % mse)
    print("Variance: %.4f" % (np.mean(Yscale[initial_train_months:]**2)))
    print("R-squared: %.4f" % (1- mse/(np.mean(Yscale[initial_train_months:]**2))))
     


In [16]:
def gen_returns(Y, P, first_pred_month, start_date='01/01/1970', freq='M', verbose=False):
    #TODO: more general version
    #take an indicator (P), a function to generate a portfolio based on P, returns (Y), return portfolio returns

    global R
    R = np.zeros(P.shape[0])
    nrows, ncols = P.shape
    numstocks = 6 # top quintile (and bottom)

    indcount = [0 for response in responses]
    longcount = [0 for response in responses]
    shortcount = [0 for response in responses]
        
    for month_index in range(first_pred_month, nrows):
        # get indexes of sorted smallest to largest
        # leftmost 6
        # ignore nan
        short_sort_array = [999999 if np.isnan(x) else x for x in P[month_index]]
        select_array = np.argsort(short_sort_array)
        short_indexes = select_array[:numstocks]
        # rightmost 6
        long_sort_array = [-999999 if np.isnan(x) else x for x in P[month_index]]
        select_array = np.argsort(long_sort_array)
        long_indexes = select_array[-numstocks:]
        # compute equal weighted long/short return
        return_month = month_index + 1
        if verbose:
            print("Longs for month %d: %s" %(return_month, str([(l,P[month_index, l]) for l in long_indexes])))
            print("Shorts for month %d: %s" %(return_month, str([(l,P[month_index, l]) for l in short_indexes])))
            
        if return_month < nrows: # last row has a prediction for following month but no following month
            R[return_month] = np.mean(X[return_month, long_indexes])/2 - np.mean(X[return_month, short_indexes])/2
            # count occurrences of each industry
            for i in short_indexes:
                indcount[i]+=1
                shortcount[i]+=1
            for i in long_indexes:
                indcount[i]+=1
                longcount[i]+=1

    for response in responses:
        i = response_reverse_dict[response]
        print("%s: long %d times, short %d times, total %d times" % (response, longcount[i], shortcount[i], indcount[i]))
        
    results = R[first_pred_month:]

    index = pd.date_range(start_date,periods=results.shape[0], freq=freq)
    perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
    perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)

    stats = perfdata['Equity'].calc_stats()

    retframe = pd.DataFrame([stats.stats.loc['start'],
                             stats.stats.loc['end'],
                             stats.stats.loc['cagr'],
                             stats.stats.loc['yearly_vol'],
                             stats.stats.loc['yearly_sharpe'],
                             stats.stats.loc['max_drawdown'],
                             ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=564, annualize=False),
                            ],
                            index = ['start',
                                     'end',
                                     'cagr',
                                     'yearly_vol',
                                     'yearly_sharpe',
                                     'max_drawdown',
                                     'sortino',
                                    ],
                            columns=['Value'])   
    return retframe


In [17]:
def create_model():
    def myclosure():
        return mord.OrdinalRidge()
    return myclosure

run_backtest(X, Y, create_model(), coef_dict='all', initial_train_months=FIRST_TRAIN_MONTHS, step=1,
             standardscale=False, verbose=False)
gen_returns(X, P, FIRST_PREDICT_MONTH, verbose=False)




fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(1, 30)
fitting on (121, 30)
 vs (121,)
predicting on(

.fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on(1, 30)
fitting on (127, 30)
 vs (127,)
predicting on

predicting on(1, 30)
fitting on (132, 30)
 vs (132,)
predicting on(1, 30)
fitting on (132, 30)
 vs (132,)
predicting on(1, 30)
fitting on (132, 30)
 vs (132,)
predicting on(1, 30)
fitting on (132, 30)
 vs (132,)
predicting on(1, 30)
fitting on (132, 30)
 vs (132,)
predicting on(1, 30)
fitting on (132, 30)
 vs (132,)
predicting on(1, 30)
fitting on (132, 30)
 vs (132,)
predicting on(1, 30)
fitting on (132, 30)
 vs (132,)
predicting on(1, 30)
fitting on (132, 30)
 vs (132,)
predicting on(1, 30)
fitting on (132, 30)
 vs (132,)
predicting on(1, 30)
.fitting on (133, 30)
 vs (133,)
predicting on(1, 30)
fitting on (133, 30)
 vs (133,)
predicting on(1, 30)
fitting on (133, 30)
 vs (133,)
predicting on(1, 30)
fitting on (133, 30)
 vs (133,)
predicting on(1, 30)
fitting on (133, 30)
 vs (133,)
predicting on(1, 30)
fitting on (133, 30)
 vs (133,)
predicting on(1, 30)
fitting on (133, 30)
 vs (133,)
predicting on(1, 30)
fitting on (133, 30)
 vs (133,)
predicting on(1, 30)
fitting on (133, 30)
 vs

.fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on(1, 30)
fitting on (138, 30)
 vs (138,)
predicting on

fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(1, 30)
fitting on (143, 30)
 vs (143,)
predicting on(

...

fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
fitting on (666, 30)
 vs (666,)
predicting on(1, 30)
.fitting on (667, 30)
 vs (667,)
predicting on(1, 30)
fitting on (667, 30)
 vs (667,)
predicting on(1, 30)
fitting on (667, 30)
 vs (667,)
predicting on(1, 30)
fitting on (667, 30)
 vs (667,)
predicting on(1, 30)
fitting on (667, 30)
 vs (667,)
predicting on(1, 30)
fitting on (667, 30)
 vs (667,)
predicting on(1, 30)
fitting on (667, 30)
 vs (667,)
predicting on

fitting on (671, 30)
 vs (671,)
predicting on(1, 30)
fitting on (671, 30)
 vs (671,)
predicting on(1, 30)
fitting on (671, 30)
 vs (671,)
predicting on(1, 30)
fitting on (671, 30)
 vs (671,)
predicting on(1, 30)
fitting on (671, 30)
 vs (671,)
predicting on(1, 30)
fitting on (671, 30)
 vs (671,)
predicting on(1, 30)
.fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on(1, 30)
fitting on (672, 30)
 vs (672,)
predicting on

.fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on(1, 30)
fitting on (677, 30)
 vs (677,)
predicting on

fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(1, 30)
fitting on (682, 30)
 vs (682,)
predicting on(

,Value
start,1970-01-31 00:00:00
end,2016-12-31 00:00:00
cagr,0.016868
yearly_vol,0.0581211
yearly_sharpe,0.314753
max_drawdown,-0.187294
sortino,0.149221


In [18]:
# map each X and Y value to its rank
Yargsort = np.argsort(Y)
Xargsort = np.argsort(X)

print(Y[0])
print(Yargsort[0])
print(Y[0, Yargsort[0]])

Yindices = np.zeros_like(Y, dtype=np.int)
Xindices = np.zeros_like(X, dtype=np.int)

for row in range(Yindices.shape[0]):
    for col in range(Yindices.shape[1]):
        Yindices[row, Yargsort[row, col]] = col
        Xindices[row, Xargsort[row, col]] = col
        
print(Yindices[0])


[ -4.49  -5.71  -2.05   1.21  -5.47  -7.84  -8.53  -6.68 -10.03  -4.77
  -6.67  -9.38  -4.42 -12.3  -11.71  -5.03  -3.81  -7.91  -7.82  -2.4
   0.62  -6.18  -7.93  -9.41  -4.31  -5.33  -6.09 -10.08  -4.68  -3.98]
[13 14 27  8 23 11  6 22 17  5 18  7 10 21 26  1  4 25 15  9 28  0 12 24
 29 16 19  2 20  3]
[-12.3  -11.71 -10.08 -10.03  -9.41  -9.38  -8.53  -7.93  -7.91  -7.84
  -7.82  -6.68  -6.67  -6.18  -6.09  -5.71  -5.47  -5.33  -5.03  -4.77
  -4.68  -4.49  -4.42  -4.31  -3.98  -3.81  -2.4   -2.05   0.62   1.21]
[21 15 27 29 16  9  6 11  3 19 12  5 22  0  1 18 25  8 10 26 28 13  7  4
 23 17 14  2 20 24]


In [19]:
print(X[0])
print(Xargsort[0])
print(X[0, Xargsort[0]])
print(Xindices[0])



[ 2.01  0.35 -3.02  1.64  7.29  0.67  1.87 -1.97  3.08  0.74  0.72  2.18
  3.11  5.53  5.28 -4.88  4.4   2.07  3.93  1.79  3.97 -1.96  0.67  1.46
  2.03 -0.3   2.41  2.71  0.15 -6.5 ]
[29 15  2  7 21 25 28  1  5 22 10  9 23  3 19  6  0 24 17 11 26 27  8 12
 18 20 16 14 13  4]
[-6.5  -4.88 -3.02 -1.97 -1.96 -0.3   0.15  0.35  0.67  0.67  0.72  0.74
  1.46  1.64  1.79  1.87  2.01  2.03  2.07  2.18  2.41  2.71  3.08  3.11
  3.93  3.97  4.4   5.28  5.53  7.29]
[16  7  2 13 29  8 15  3 22 11 10 19 23 28 27  1 26 18 24 14 25  4  9 12
 17  5 20 21  6  0]


In [20]:
c = mord.LogisticIT() #Default parameters: alpha=1.0, verbose=0, maxiter=10000
c.fit(np.array([[0,0,0,1],[0,1,0,0],[1,0,0,0]]), np.array([1,2,3]))
print(c.predict(np.array([0,0,0,1])))
print(c.predict(np.array([0,1,0,0])))
print(c.predict(np.array([1,1,0,0])))

[1]
[2]
[3]


In [21]:
def create_model():
    def myclosure():
        return mord.LogisticSE()
    return myclosure

run_backtest(X, Yindices, mord.LogisticAT, coef_dict='all', initial_train_months=241, step=1,
             standardscale=False, verbose=False)
gen_returns(X, P, 241, verbose=False)



fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)
predicting on(1, 30)
fitting on (241, 30)
 vs (241,)


ValueError: Values in y must be [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26], instead got [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 25 26 27]

In [ ]:
# double check results_post_LASSO
#model = LinearRegression()
#R = run_backtest(X, Y, model, coef_dict_paper, startmonth=STARTMONTH, summary=False)
results_post_LASSO = R[FIRST_PREDICT_MONTH:]
print(len(results_post_LASSO))
#print(results_post_LASSO)
print(np.mean(results_post_LASSO))
print(np.std(results_post_LASSO) * np.sqrt(12))
print(np.prod(1 + results_post_LASSO / 100))
print(np.prod(1 + results_post_LASSO / 100) ** (12.0/results_post_LASSO.shape[0]))-1

In [ ]:
# run performance chart
perf_post_LASSO = 100 * np.cumprod(1 + results_post_LASSO / 100)

def mychart(args, names=None):
    x_coords = np.linspace(1970, 2016, args[0].shape[0])
    
    plotdata = []
    for i in range(len(args)):
        tracelabel = "Trace %d" % i
        if names:
                tracelabel=names[i]
        plotdata.append(Scatter(x=x_coords,
                                y=args[i].reshape(-1),
                                mode = 'line',
                                name=tracelabel))    

    layout = Layout(
        autosize=False,
        width=600,
        height=480,
        yaxis=dict(
            type='log',
            autorange=True
        )
    )
    
    fig = Figure(data=plotdata, layout=layout)
    
    return iplot(fig)
    
mychart([perf_post_LASSO],["Post-LASSO"])

In [ ]:
# pass coef_dict as None
# fit_predict will do subset selection at each timestep using data it trains on
run_backtest(X, Y, create_model(), coef_dict=None, initial_train_months=FIRST_TRAIN_MONTHS)
gen_returns(X, P, FIRST_PREDICT_MONTH)


In [ ]:
results_LASSO_each_timestep = R[FIRST_PREDICT_MONTH:]
perf_LASSO_each_timestep = 100 * np.cumprod(1 + results_LASSO_each_timestep / 100)
mychart([perf_LASSO_each_timestep])

In [ ]:
# pass coef_dict as 'all'
# fit_predict will use all predictors (no subset selection)
run_backtest(X, Y, create_model(), coef_dict='all', initial_train_months=FIRST_TRAIN_MONTHS)
gen_returns(X, P, FIRST_PREDICT_MONTH)


In [ ]:
results_OLS = R[FIRST_PREDICT_MONTH:]
perf_OLS = 100 * np.cumprod(1 + results_OLS / 100)
mychart([perf_OLS])

In [ ]:
mychart([perf_post_LASSO, perf_LASSO_each_timestep, perf_OLS],["Post-LASSO", "LASSO each timestep", "OLS"])

In [ ]:
def walkforward_xval (X, Y, create_model, coef_dict="all", n_splits=5):

    start = time.time()

    # generate k-folds
    kf = KFold(n_splits=n_splits)
    kf.get_n_splits(X)
    last_indexes = []
    for train_index, test_index in kf.split(X):
        # use test_index as last index to train
        last_index = test_index[-1] + 1
        last_indexes.append(last_index)
    print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))

    print("%s Starting training" % (time.strftime("%H:%M:%S")))
    model = create_model()
    avg_bests = []
    for i in range(1, n_splits-1):

        models = []
        losses = []
        scores = []
        count = 0        
        # skip kfold 0 so you start with train 2x size of eval set
        last_train_index = last_indexes[i]
        last_xval_index = last_indexes[i+1]

        # set up train, xval
        # train from beginning to last_train_index        
        print("Training indexes 0 to %d" % (last_train_index-1))
        X_fit = X[:last_train_index]
        Y_fit = Y[:last_train_index]
        # xval from last_train_index to last_xval_index
        print("Cross-validating indexes %d to %d" % (last_train_index, last_xval_index -1 ))
        X_xval = X[last_train_index:last_xval_index]
        Y_xval = Y[last_train_index:last_xval_index]

        if coef_dict is None:
            print("Performing LASSO subset selection on training set")
            coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'), verbose=False)
        elif coef_dict == 'all':
            # if coef_dict == "all" use all predictors for each response
            coef_dict = {}
            for response in responses:
                coef_dict[response]=predictors
                
        mse_list = []
        
        for response in responses:
            predcols = [predictor_reverse_dict[indstr] for indstr in coef_dict[response]]
            if len(predcols) == 0:
                continue
            responsecol = response_reverse_dict[response]
            
            fit = model.fit(X_fit[:,predcols], Y_fit[:,responsecol])
            # evaluate ... run prediction, calc MSE by industry, and average
            y_xval_pred = fit.predict(X_xval[:,predcols])
            mse_list.append(mean_squared_error(Y_xval[:,i], y_xval_pred))
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training" % (time.strftime("%H:%M:%S")))
            sys.stdout.flush()             
        # mean mse over industry ys for this fold
        xval_score = np.mean(np.array(mse_list))            

        # choose model with lowest xval loss
        print ("\n%s Xval MSE %f" % (time.strftime("%H:%M:%S"), xval_score))
        avg_bests.append(xval_score)
    
    print ("Last Xval loss %f" % (xval_score))
    # mean over folds
    avg_loss = np.mean(np.array(avg_bests))
    print ("Avg Xval loss %f" % avg_loss)
    print("--------------------------------------------------------------------------------")
    return (avg_loss, model)


In [ ]:
walkforward_xval (X, Y, LinearRegression, coef_dict=coef_dict)


In [ ]:
# closure to wrap model creation function - return a function to create model
def create_model(n_hidden_layers, layer_size, reg_penalty):
    def myclosure():
        return MLPRegressor(hidden_layer_sizes=tuple([layer_size]*n_hidden_layers),
                            alpha=reg_penalty,
                            activation='tanh',
                            max_iter=10000, 
                            tol=1e-10,
                            solver='lbfgs')
    return myclosure

walkforward_xval (X, Y, create_model(3,2,1.0), coef_dict=coef_dict)
# result is a little worse than LinearRegression

In [ ]:
MODELPREFIX = "MLP"

n_hiddens = [1, 2, 3]
layer_sizes = [1, 2, 4, 8]
reg_penalties = [0.0, 0.001, 0.01, 0.1, 1]
hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))

experiments = {}

Xscale = X.copy()
Yscale = Y.copy()

# minmaxscale each row
for i in range(Xscale.shape[0]):
    Xscale[i] = Xscale[i] - np.min(Xscale[i])
    Xscale[i] = Xscale[i]/np.max(Xscale[i])

for i in range(Yscale.shape[0]):
    Yscale[i] = Yscale[i] - np.min(Yscale[i])
    Yscale[i] = Yscale[i]/np.max(Yscale[i])
        
for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty)
    print("%s n_hidden_layers = %d, hidden_layer_size = %d, reg_penalty = %.6f" % 
          (time.strftime("%H:%M:%S"), n_hidden_layers, layer_size, reg_penalty))
    experiment_model = create_model(n_hidden_layers,layer_size,reg_penalty)
    score, model = walkforward_xval (X, Y, experiment_model, coef_dict=coef_dict)
    experiments[key] = score


In [ ]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]
 
lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "loss"])
lossframe.sort_values(['loss'])

In [ ]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['loss'].mean())


In [ ]:
pd.DataFrame(lossframe.groupby(['layer_size'])['loss'].mean())


In [ ]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['loss'].mean())


In [ ]:
def plot_matrix(lossframe, x_labels, y_labels, x_suffix="", y_suffix=""):

    pivot = lossframe.pivot_table(index=[y_labels], columns=[x_labels], values=['loss'])
#    print(pivot)
    # specify labels as strings, to force plotly to use a discrete axis
#    print(pivot.columns.levels[1]).values
#    print(lossframe[x_labels].dtype)
    
    if lossframe[x_labels].dtype == np.float64 or lossframe[x_labels].dtype == np.float32:
        xaxis = ["%f %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    else:
        xaxis = ["%d %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    if lossframe[y_labels].dtype == np.float64 or lossframe[y_labels].dtype == np.float32:
        yaxis = ["%f %s" % (i, y_suffix) for i in pivot.index.values]
    else:
        yaxis = ["%d %s" % (i, y_suffix) for i in pivot.index.values]
        
#    print(xaxis, yaxis)
    """plot a heat map of a matrix"""
    chart_width=640
    chart_height=480
    
    layout = Layout(
        title="%s v. %s" % (x_labels, y_labels),
        height=chart_height,
        width=chart_width,     
        margin=dict(
            l=150,
            r=30,
            b=120,
            t=100,
        ),
        xaxis=dict(
            title=x_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
        yaxis=dict(
            title=y_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
    )
    
    data = [Heatmap(z=pivot.values,
                    x=xaxis,
                    y=yaxis,
                    colorscale=[[0, 'rgb(0,0,255)', [1, 'rgb(255,0,0)']]],
                   )
           ]

    fig = Figure(data=data, layout=layout)
    return iplot(fig, link_text="")

plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" layers", y_suffix=" units")




In [ ]:
plot_matrix(lossframe, "n_hidden_layers", "reg_penalty", x_suffix=" layers", y_suffix=" p")


In [ ]:
plot_matrix(lossframe, "reg_penalty", "layer_size", x_suffix=" p", y_suffix="units")


In [ ]:
# 1-unit layers is not really a NN but anyway let's see how it does
experiment_model = create_model(3,1,0.01)

run_backtest(X, Y, experiment_model, initial_train_months=FIRST_TRAIN_MONTHS, minmaxscale=False)
gen_returns(X, P, FIRST_PREDICT_MONTH, verbose=False)
